In [1]:
import pandas as pd
import bamboolib
import numpy as np
import ast

In [2]:
def norm(x):
    return (1/np.log(x))

#### Get the graines, their friends & their followers

In [3]:
# Load all the followers of the graines
foll = pd.read_csv('data/followers_graines_version_2021_09_21.csv', index_col = [0], low_memory=False).reset_index()
foll = foll[['twitter_handle', 'follower_id']]

# Create the dictionary of graines and their followers
group_foll = foll.groupby('twitter_handle')['follower_id'].count().rename('count_followers').reset_index()
dict_follo = dict(zip(group_foll['twitter_handle'], group_foll['count_followers']) )

# Load of the friends of the graines
fri = pd.read_csv('data/friends_graines.csv.gz', index_col = [0], low_memory=False).reset_index()
fri = fri[['twitter_handle', 'friend_id']]

# Create the dictionary of graines and their friends
group_fri = fri.groupby('twitter_handle')['friend_id'].count().rename('count_friends').reset_index()
dict_fri = dict(zip(group_fri['twitter_handle'], group_fri['count_friends']) )

#### Get the missing values of the dataset

In [4]:
# Load the dataset of the candidates
data_candidate = pd.read_csv('data/data_ready.csv', index_col = [0])
data = data_candidate[['user_id', 'followers', 'friends', 'count_graines_in_friends', 'graines_in_friends']]
data['graines_in_friends'] = data['graines_in_friends'].apply(lambda x: ast.literal_eval(x))

In [5]:
# The followers information of the original dataset is missing
foll_grain = pd.merge(data, foll, left_on = 'user_id', right_on = 'follower_id').drop('follower_id', axis=1)
foll_grain = foll_grain.groupby('user_id')['twitter_handle'].agg(['count', list])
foll_grain.columns = ['count_graines_in_followers', 'graines_in_followers']
foll_grain = foll_grain.reset_index()

# remerge with the datase
new_data = pd.merge(foll_grain, data, on = 'user_id')

### Followers

In [6]:
new_data_follo = new_data.explode('graines_in_followers')
new_data_follo['total_grain_follo'] = new_data_follo['graines_in_followers'].apply(lambda x: dict_follo.get(x))

# get the sum of the total followers of graine follows by an indivual
new_data_follo = new_data_follo.groupby('user_id')['total_grain_follo'].sum().reset_index()

# Normalize by the formula
new_data_follo['normalize_follo'] = new_data_follo['total_grain_follo'].apply(lambda x: norm(x))

### Friends

In [7]:
new_data_fri = new_data.explode('graines_in_friends')
new_data_fri['total_grain_fri'] = new_data_fri['graines_in_friends'].apply(lambda x: dict_fri.get(x))

# get the sum of the total followers of graine follows by an indivual
new_data_fri = new_data_fri.groupby('user_id')['total_grain_fri'].sum().reset_index()

# Normalize by the formula
new_data_fri['normalize_friends'] = new_data_fri['total_grain_fri'].apply(lambda x: norm(x))

### Final concat

In [8]:
concat_score = pd.merge(new_data_follo, new_data_fri, on = 'user_id')
concat_score = pd.merge(concat_score, new_data, on = 'user_id')

In [10]:
concat_score['prop_graine_friends'] = concat_score['count_graines_in_friends']/concat_score['friends']
concat_score['prop_graine_followers'] = concat_score['count_graines_in_followers']/concat_score['followers']

In [92]:
final = concat_score[['count_graines_in_followers', 'prop_graine_followers',
                    'count_graines_in_friends', 'prop_graine_friends', 
                    'normalize_friends', 'normalize_follo']]



final = final.fillna(final.mean())
final = final.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
final = np.array(final)
np.save('embeddings/topo.npy', final)

In [102]:
y = list(data_candidate.label)

In [105]:
import autosklearn.classification
cls = autosklearn.classification.AutoSklearnClassifier()
cls.fit(X_train, y_train)
predictions = cls.predict(X_test)

ModuleNotFoundError: No module named 'autosklearn'

In [106]:
!pip install autosklearn

ERROR: Could not find a version that satisfies the requirement autosklearn (from versions: none)
ERROR: No matching distribution found for autosklearn
